# SC1015 Lab REP2 Team 4 DSAI Mini-Project

### Sricharan Balasubramanian, Travis Tan Hai Shuo, Joanna Wu Haoyue

In light of our upcoming exchange in UC Berkeley, the 3 of us are excited to go on many roadtrips to explore the beautiful state of California. However, we are concerned about the road safety there and want to know which streets are most prone to accidents, so we can be more cautious.

Upon finding the Kaggle dataset of "US Accidents" that contained data on road accidents all across US, we decided to scope down this dataset to only accidents that happened in the state of California. Coincidentally, with a quick visualisation of the dataset, we realised California is the state with the highest number of road accidents, which inspired a greater goal and use case.

Thus, in this project we will be using this dataset to find out the times of day or night where particular streets are most accident prone. We also want to find correlation between weather conditions and accidents. With these insights, emergency services like firefighters and hospitals can have an early warning system to help them allocate rescue resources optimally, reducing the fatality of road accidents. Additionally, these insights can be integrated into GPS systems to alert drivers when they are driving on certain streets at accident-prone times or weather conditions, minimising road accidents.

In [ ]:
!pip install geopandas
!pip install contextily
!pip install shapely
!pip install folium
!pip install imbalanced-learn xgboost
!pip install pandoc

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
import time
import geopandas as gpd
import contextily as ctx
from shapely.geometry import Point
from matplotlib.ticker import MaxNLocator
import folium
from folium.plugins import HeatMap
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import Ridge, LogisticRegression
from tqdm import tqdm
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,classification_report, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

In [ ]:
df = pd.read_csv('US_Accidents_March23_Cleaned.csv')

In [ ]:
df.head(500)

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df.describe(include='all')

### Data Pre-processing

We will prepare the data by performing data cleaning to suit our problem statement. This begins with feature reduction on the California dataset, to improve model performance and reduce complexity. 

We will eliminate columns that are not necessary for our analysis, such as columns with only 1 unique value (e.g. 'country' and 'state'columns, because all accidents happen in the same country and state). Likewise, for Turning_Loop where every value is False, there is no value added with purely negative data. We will also eliminate columns that are irrelevant to the problem, like timezone and the various twilights, as they are just alternative ways to tell time. 

We will also analyse the duration of accident using (end_time - start_time) to see if end_time can be dropped.

In [ ]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')
df['End_Time'] = pd.to_datetime(df['End_Time'], errors='coerce')

In [ ]:
df['Duration'] = df['End_Time'] - df['Start_Time']
df['Duration_Minutes'] = df['Duration'].dt.total_seconds() / 60

Since some of the accident durations are days, we will remove them from this visualisation as the focus of our project is not on how accidents affect traffic.

In [ ]:
filtered_df = df[df['Duration_Minutes'] <= 300]

plt.figure(figsize=(10, 5))
sb.histplot(filtered_df['Duration_Minutes'], bins=50, kde=True)
plt.title("Filtered Distribution of Accident Durations (<= 300 mins)")
plt.xlabel("Duration (minutes)")
plt.ylabel("Frequency")
plt.show()

We observe that most accident durations is about 25 minutes, which we deem to be too short to be relevant to our dataset and problem statement. Hence, we justify removing End_Time (and therefore End_Lat, End_Lng too) from our dataset.

In [ ]:
columns_to_drop = [
    'ID', 'Source', 'End_Time','End_Lat', 'End_Lng', 'Country', 'State', 'Timezone', 'Turning_Loop','Airport_Code', 'Weather_Timestamp',
    'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Duration', 'Duration_Minutes'
]
df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
df.info()

We will now convert this dataset's features into their appropriate types, making it easier to handle the data and help the ML models better understand the features. This is important for datetime and categorical features.

We realise an important step is to split the Start_Time into date and time separately, because our problem statement considers time to be more important than date.

In [ ]:
# Convert object columns to category
object_cols = df.select_dtypes(include=['object']).columns
df[object_cols] = df[object_cols].astype('category')

# Specifically convert 'Description' to string
df["Description"] = df["Description"].astype('string')

# Start_Time has already been converted into datetime earlier, but here we will split it into date and time separately

# Split Start_Time into separate features
df['Start_Date'] = df['Start_Time'].dt.date
df['Start_Hour'] = df['Start_Time'].dt.hour
df['Start_Minute'] = df['Start_Time'].dt.minute
df['Accident_Time'] = df['Start_Hour'] + df['Start_Minute'] / 60

# Update column type lists AFTER cleaning
datetime_cols = df.select_dtypes(include=['datetime64[ns]']).columns.tolist()
cat_cols = df.select_dtypes(include=['category']).columns.tolist()
bool_cols = df.select_dtypes(include=['bool']).columns.tolist()
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
string_cols = df.select_dtypes(include=['string']).columns.tolist()

These are the converted data types.

In [ ]:
df.info()

After feature reduction and keeping only columns we deemed as relevant, now we will handle any NULL values present in the dataset. Let's look at the spread of NULL values within the dataset.





In [ ]:
null_count = df.isnull().sum()

total_rows = len(df)
null_percentage = (null_count / total_rows) * 100

null_df = pd.DataFrame({'Null Count': null_count, 'Null Percentage': null_percentage})
print(null_df)

We noticed approximately 10% of the dataset contains missing values in the Start_Time column, which might be a result of improper date time format that failed to convert, resulting in NaT (Not a time) NULL value replacing the data point. Since Start_Time is a critical variable used to derive several temporal features such as accident time, rows lacking this information are unsuitable for time-based analysis, but 10% is a considerable amount of data to be dropped.

We decided to visualise the distribution of streeets across the dataset to observe if the rows with NULL accident time will skew the results heavily if removed. For example, if a street has disproportionately higher accidents with no time than ones recorded with time, then removing the NULL times would impact the ML results of that street. We will select the streets with the most number of accidents to plot.



In [ ]:
df_with_time = df[df['Start_Time'].notnull()]
df_without_time = df[df['Start_Time'].isnull()]

# Count accidents per street for each group
street_counts_with_time = df_with_time['Street'].value_counts()
street_counts_without_time = df_without_time['Street'].value_counts()

# Combine counts into a DataFrame
street_counts = pd.DataFrame({
    'With Start_Time': street_counts_with_time,
    'Without Start_Time': street_counts_without_time
}).fillna(0)

# Normalise to get proportions
street_counts_norm = street_counts.div(street_counts.sum(axis=0), axis=1)

# Select top N streets by total accidents
top_n = 10
top_streets = street_counts.sum(axis=1).nlargest(top_n).index
street_counts_top = street_counts_norm.loc[top_streets]

# Plot
street_counts_top.plot(kind='bar', figsize=(12, 6))
plt.title('Proportion of Accidents by Street: With vs. Without Start_Time')
plt.xlabel('Street')
plt.ylabel('Proportion of Accidents')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Since the proportions of accidents across streets are similar between the two groups, dropping rows with missing Start_Time is unlikely to bias our analysis. We also believe that imputing the NULL times with an average time would not be the best way to handle the data, and thus decided on dropping rows with missing Start_Time.

In [ ]:
df.dropna(subset=['Start_Time'], inplace=True)

In [ ]:
df.info()

To address the other NULL values, they will be broken down into 3 groups: 
1. Crticial columns with low NULLs/NULL percentages: These rows will be dropped as the change is negligible but will greatly improve quality of dataset
2. Numerical weather features (2-9% missing values): These rows will be filled with median values to reflect the central tendency of these weather conditions
3. Columns with high NULLs/NULL percentages (Wind_Chill and Precipitation): Wind_Chill will be dropped as with such a high NULL percentage, it is often redundant and comes hand in hand with attribute from another column such as Temperature. 

Precipitation NULLs will be inputed with 0 but not dropped, because rain is an important weather condition. If precipiation were a factor of the accident, we assume it would have been documented. Thus, empty cells for precipiation implies rain was not a contributive factor and we assume NULL cells will be replaced with 0. Disclaimer: precipitation is often not measured precisely as snow may not be counted as Precipitation.

We will ignore Description column.

In [ ]:
print("Missing values BEFORE cleaning:")
print(df.isnull().sum()[df.isnull().sum() > 0].sort_values(ascending=False))

# dealing with group 1
df.dropna(subset=['Street', 'City'], inplace=True)

# dealing with group 2
weather_num_cols = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']
for col in weather_num_cols:
    df[col].fillna(df[col].median(), inplace=True)

# handling missing Zipcode
if 'Unknown' not in df['Zipcode'].cat.categories:
    df['Zipcode'] = df['Zipcode'].cat.add_categories('Unknown')
df['Zipcode'] = df['Zipcode'].fillna('Unknown')


# Fill categorical/object-based weather columns with 'Unknown'
if 'Wind_Direction' in df.columns and df['Wind_Direction'].dtype.name == 'category':
    if 'Unknown' not in df['Wind_Direction'].cat.categories:
        df['Wind_Direction'] = df['Wind_Direction'].cat.add_categories('Unknown')
    df['Wind_Direction'] = df['Wind_Direction'].fillna('Unknown')

if 'Weather_Condition' in df.columns and df['Weather_Condition'].dtype.name == 'category':
    if 'Unknown' not in df['Weather_Condition'].cat.categories:
        df['Weather_Condition'] = df['Weather_Condition'].cat.add_categories('Unknown')
    df['Weather_Condition'] = df['Weather_Condition'].fillna('Unknown')

df.drop(columns=['Wind_Chill(F)'], inplace=True) 
df['Precipitation(in)'].fillna(0, inplace=True)  # assumes most missing = no rain


# Optional: Fill less important columns with default (for modeling)
optional_bool_cols = df.select_dtypes(include='bool').columns
df[optional_bool_cols] = df[optional_bool_cols].fillna(False)

# Check again after cleaning
print("\nMissing values AFTER cleaning:")
print(df.isnull().sum()[df.isnull().sum() > 0])


We have successfully handled NULL values in our dataset!

### Distribution Analysis & Normalisation

Now, looking at how some columns have many unique values (continuous numericial), we will normalise the data to remove outliers as they can skew data analysis and affect the reliability of ML models like KNN and clustering.

We will begin with a visualisation of potential outliers in the dataset.

In [ ]:
def plot_numeric_features(df, numeric_features):
    df[numeric_features] = df[numeric_features].apply(pd.to_numeric, errors='coerce')
    df[numeric_features] = df[numeric_features].fillna(0)
    num_cols = len(numeric_features)
    fig, axs = plt.subplots(num_cols, 2, figsize=(12, num_cols*6))

    for i, feature in enumerate(numeric_features):
        # Histogram
        axs[i, 0].hist(df[feature], bins=20, color='skyblue', edgecolor='black')
        axs[i, 0].set_title(f'{feature} Histogram')
        axs[i, 0].set_xlabel(feature)
        axs[i, 0].set_ylabel('Frequency')

        # Boxplot
        axs[i, 1].boxplot(df[feature], vert=True)
        axs[i, 1].set_title(f'{feature} Boxplot')
        axs[i, 1].set_xlabel(feature)
        
    plt.tight_layout()
    plt.show()

numeric_features = ['Distance(mi)', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 
                    'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']
plot_numeric_features(df, numeric_features)


Outliers are detected in precipiation, wind_speed, visbility and distance. Possible explanations would be that most days go with 0 rain, and crashes often occur nearby. Either ways, the data can be normalised to minimise the skew.

In [ ]:
def plot_boolean_features(df, bool_features):
    # Calculate the number of rows and columns for subplots
    num_features = len(bool_features)
    num_rows = num_features // 2 + num_features % 2
    num_cols = 2

    # Create subplots with rectangular shape
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 30))

    # Flatten the axes array to make it easier to iterate
    axes = axes.flatten()

    # Plot boolean counts for specified features
    for i, feature in enumerate(bool_features):
        counts = df[feature].value_counts()
        counts.plot(kind='bar', ax=axes[i])
        axes[i].set_title(f'Boolean counts for {feature}')
        axes[i].set_xlabel('Value')
        axes[i].set_ylabel('Count')

    # Hide empty subplots
    for j in range(num_features, num_rows * num_cols):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

plot_boolean_features(df, bool_cols)

In all the above graphs, false values are more frequent than true values.

In [ ]:
for column in df[bool_cols].columns:
    true_percentage = df[column].mean() * 100
    false_percentage = 100 - true_percentage

    print(column)
    print(f"Percentage of True: {true_percentage:.2f}%")
    print(f"Percentage of False: {false_percentage:.2f}%")
    print()


Only Traffic_Signal, Junction, and Crossing have True values over 3% of all values.

In [ ]:
def plot_top_categories(df, cat_features):
    # Calculate the number of rows and columns for subplots
    num_features = len(df[cat_features].columns)
    num_rows = num_features // 2 + num_features % 2
    num_cols = 2

    # Create subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 50))

    # Flatten the axes array to make it easier to iterate
    axes = axes.flatten()

    # Plot top categories for each categorical feature
    for i, column in enumerate(df[cat_features].columns):
        top_categories = df[column].value_counts().nlargest(50)
        top_categories.plot(kind='bar', ax=axes[i])
        axes[i].set_title(f'Top categories for {column}')
        axes[i].set_xlabel('Category')
        axes[i].set_ylabel('Count')

    # Hide empty subplots
    for j in range(num_features, num_rows * num_cols):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

plot_top_categories(df, cat_cols)

Now we normalise and remove outliers.

In [ ]:
# Select only numeric columns excluding 'Severity'
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = [col for col in numeric_cols if col != 'Severity']  # Remove 'Severity' column

# Normalize the remaining numeric columns
scaler = MinMaxScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Outlier removal
Q1 = df[numeric_cols].quantile(0.25)
Q3 = df[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

# Filter out rows outside 1.5 * IQR range, excluding 'Severity' column
df = df[~((df[numeric_cols] < (Q1 - 1.5 * IQR)) | (df[numeric_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]


### Correlation Analysis - Weather Conditions

We will perform correlation analysis to identify the most correlated weather conditions with severity, and illustrate the relationships between various conditions

In [ ]:
num_cols = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Wind_Speed(mph)']

scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

plt.figure(figsize=(10, 6))
sb.heatmap(df[num_cols + ['Severity']].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap with Severity")
plt.show()

We will now perform feature merging to combine similar features. For example, we will combine Weather_Conditions as there are too many unique values inside, amany of which are overly-specified and semantically similar. This will reduce the complexity of our ML model, and improve its efficiency.

To address this, we will map specific weather conditions into broader categories, and create binary flags for important weather conditions (e.g. Is_Windy, Is_Wet) to improve model interpretability and performance.

In [ ]:
def simplify_weather(condition):
    condition = condition.lower()
    if 'rain' in condition or 'drizzle' in condition or 'shower' in condition:
        return 'Rain'
    elif 'snow' in condition or 'sleet' in condition:
        return 'Snow'
    elif 'fog' in condition or 'mist' in condition or 'haze' in condition or 'smoke' in condition:
        return 'Fog'
    elif 'storm' in condition or 'thunder' in condition or 'tstorm' in condition:
        return 'Storm'
    elif 'clear' in condition or 'sun' in condition:
        return 'Clear'
    elif 'cloud' in condition or 'overcast' in condition:
        return 'Cloudy'
    elif 'wind' in condition or 'breezy' in condition or 'gusty' in condition:
        return 'Windy'
    else:
        return 'Other' # how many are classified under other?

df['Weather_Simple'] = df['Weather_Condition'].astype(str).apply(simplify_weather)

#to see other
# Count how many rows were classified as 'Other'
other_count = df[df['Weather_Simple'] == 'Other'].shape[0]
print(f"Number of rows classified as 'Other': {other_count}")

# See the unique Weather_Condition values that ended up as 'Other'
other_conditions = df[df['Weather_Simple'] == 'Other']['Weather_Condition'].unique()
print("Unique Weather_Condition values classified as 'Other':")
print(other_conditions)


Creating binary flags to capture each weather effect, which helps with readability and interpretability admist mixed weather conditions.

In [ ]:
df['Is_Windy'] = df['Weather_Condition'].str.contains('Wind|Breezy|Gusty', case=False, na=False).astype(int)
df['Is_Stormy'] = df['Weather_Condition'].str.contains('Storm|Thunder|Tstorm', case=False, na=False).astype(int)
df['Is_Rainy'] = df['Weather_Condition'].str.contains('Rain|Drizzle|Shower', case=False, na=False).astype(int)
df['Is_Foggy'] = df['Weather_Condition'].str.contains('Fog|Mist|Haze|Smoke', case=False, na=False).astype(int)
df['Is_Snowy'] = df['Weather_Condition'].str.contains('Snow|Sleet', case=False, na=False).astype(int)
df['Is_Clear'] = df['Weather_Condition'].str.contains('Clear|Sunny', case=False, na=False).astype(int)

In [ ]:
df.nunique()

After normalisation and outlier removal, we realised Visibility (mi) and Precipitation (in) have been reduced to only 1 unique value, which does not provide any valuable insights. Hence, these 2 columns will be dropped from the dataset.

In [ ]:
features_to_drop = [
    'Visibility(mi)', 'Precipitation(in)'
]
df.drop(columns=features_to_drop, inplace=True)

Thus, the data has been properly processed through feature reduction, feature merging, NULL-value and outlier handling. Let's create the new DataFrame to be used for the EDA and model training.

In [ ]:
df_cleaned = df.copy()

In [ ]:
df_cleaned.info()

# Exploratory Data Analysis

We have successfully cleaned our data and will now perform exploratory data analysis. We have a few hypotheses that we wish to explore, and will do them sequentially to validate potential trends, and subsequently feed into our ML algorithms for predictions. The items we wish to explore are the following:

1. Which regions in California have the most accidents, and which

2. What trends can we identify in the frequency of accidents against the time of the day, and the day of the week?

3. Do accidents occur in all weather conditions or only in certain weather conditions?

In [ ]:
df_cleaned.head(500000)

We first start with a general plot of the entire California map with the accidents displayed, to be able to pick up some general trends.

In [ ]:
# Set plotting styles
plt.rcParams['figure.figsize'] = (10, 6)

# sample for general trend
sample_df = df_cleaned[['Start_Lat', 'Start_Lng']].dropna().sample(500000, random_state=42)

# convert to GeoDataFrame in WGS84
geometry = [Point(xy) for xy in zip(sample_df['Start_Lng'], sample_df['Start_Lat'])]
gdf = gpd.GeoDataFrame(sample_df, geometry=geometry, crs='EPSG:4326')

# project to Web Mercator
gdf = gdf.to_crs(epsg=3857)

# plot the accidents
fig, ax = plt.subplots(figsize=(12, 10))
gdf.plot(ax=ax, markersize=1, alpha=0.05, color='red')

# after plotting gdf and adding basemap, get the map bounds
xmin, xmax = ax.get_xlim()
ymin, ymax = ax.get_ylim()

# add basemap and format
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.set_title('Accident Density Map: California', fontsize=14)
ax.set_axis_off()
plt.tight_layout()
plt.show()

It is clear that the accidents are not distributed homogeneously distributed in California. We see that they are concentrated in specific regions. For example, Los Angeles and the Bay Area show clear spikes. Moreover, we see accidents along lines through the state, probably indicating long roads. We confirm this with further visualisations.

In [ ]:
# get top 20 cities by accident count and turn it into a proper DataFrame
top_cities_df = df_cleaned['City'].value_counts().head(20).reset_index()
top_cities_df.columns = ['City', 'Accident_Count']

top_cities_df = top_cities_df.sort_values('Accident_Count', ascending=True)

ax = top_cities_df.plot(kind='barh', x='City', y='Accident_Count', legend=False, color='teal', figsize=(12, 6))

# annotate each bar with the exact accident count
for index, value in enumerate(top_cities_df['Accident_Count']):
    ax.text(value, index, str(value), va='center', ha='left', color='black', fontsize=12)

plt.title("Top 20 Cities with Most Accidents")
plt.xlabel("Number of Accidents")
plt.ylabel("City")
plt.tight_layout()

plt.show()

The city distribution shows clear imbalance in distributions, and this can be used as a foundation for our analysis.

In [ ]:
# get top 20 streets by accident count and turn it into a proper DataFrame
top_streets_df = df_cleaned['Street'].value_counts().head(20).reset_index()
top_streets_df.columns = ['Street', 'Accident_Count']

# sort by 'Accident_Count' in descending order
top_streets_df = top_streets_df.sort_values('Accident_Count', ascending=True)

# slot the bar chart (in ascending order, so the highest will be at the top)
ax = top_streets_df.plot(kind='barh', x='Street', y='Accident_Count', legend=False, color='salmon', figsize=(12, 6))

# annotate each bar with the street name and the exact accident count
for index, value in enumerate(top_streets_df['Accident_Count']):
    street_name = top_streets_df['Street'].iloc[index]
    ax.text(value, index, f'{street_name}: {value}', va='center', ha='left', color='black', fontsize=12)

plt.title("Top 20 Streets with Most Accidents")
plt.xlabel("Number of Accidents")
plt.ylabel("Street")
plt.tight_layout()

plt.show()

The split by street is a bit more homogeneous, and this can potentially be a weakness as there might be very high cardinality, inhibiting trend detection in our model. We have 60000 streets, and this will pose a problem in our model later.

In [ ]:
# plot: accidents per hour (ensure hour is 0–23, sorted)
plt.figure(figsize=(10, 6))
sb.countplot(
    x='Start_Hour',
    data=df_cleaned,
    order=sorted(df_cleaned['Start_Hour'].unique()),
    palette='flare'
)
plt.title("Accidents per Hour")
plt.xlabel("Hour of Day")
plt.ylabel("Number of Accidents")
plt.xticks(ticks=range(24), labels=[str(i) for i in range(24)])  # force 0–23 x-axis
plt.show()

We see a local spike between 7AM and 9AM, and a global spike between 4PM and 7PM. These are peak hours, and clearly this is another dimension for our analysis.

In [ ]:
# extract day of week from Start_Time
df_cleaned['Day_of_Week'] = df_cleaned['Start_Time'].dt.day_name()

# plot: Accidents by Day of the Week
plt.figure(figsize=(10, 6))

sb.countplot(
    x='Day_of_Week',
    data=df_cleaned,
    order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    palette='crest'
)
plt.title("Accidents by Day of the Week")
plt.xlabel("Day of Week")
plt.ylabel("Number of Accidents")
plt.xticks(rotation=45)
plt.show()

We also see a clear distinction between weekdays and weekends. A specific spike on Fridays, possibly an indicator of more haphazard driving. However, since we are trying to identify trends across both time and location, we will cross validate these trends.

In [ ]:
# group by Street and Start_Hour and calculate the count of accidents
time_location_street = df.groupby(['Street', 'Start_Hour']).size().unstack(fill_value=0)

# calculate the total accidents in each street
street_totals = time_location_street.sum(axis=1)

# normalize the counts to proportions by dividing by the total accidents in each street
time_location_proportion_street = time_location_street.div(street_totals, axis=0)

# use top 20 streets
top20_streets = df['Street'].value_counts().head(20).index
sb.heatmap(time_location_proportion_street.loc[top20_streets], cmap="YlOrRd", annot=False)  # Remove annotations
plt.title("Proportion of Accidents by Hour in Top 20 Streets")
plt.xlabel("Hour of Day")
plt.ylabel("Street")
plt.xticks(ticks=range(0, 24), labels=[str(i) for i in range(24)])  # Ensure hour ticks are 0-23
plt.show()


In [ ]:
# group by City and Start_Hour and calculate the count of accidents
time_location = df.groupby(['City', 'Start_Hour']).size().unstack(fill_value=0)

# calculate the total accidents in each city
city_totals = time_location.sum(axis=1)

# normalize the counts to proportions by dividing by the total accidents in each city
time_location_proportion = time_location.div(city_totals, axis=0)

# use top 20 cities
top20 = df['City'].value_counts().head(20).index
sb.heatmap(time_location_proportion.loc[top20], cmap="YlOrRd", annot=False)  # Remove annotations
plt.title("Proportion of Accidents by Hour in Top 20 Cities")
plt.xlabel("Hour of Day")
plt.ylabel("City")
plt.xticks(ticks=range(0, 24), labels=[str(i) for i in range(24)])  # Ensure hour ticks are 0-23
plt.show()

Both heatmaps validate there is a trend across all the different cities and streets, justifying using these as dimensions for our core analysis.

In [ ]:
# group by city and day of week, count accidents
df_cleaned['Day_of_Week'] = df_cleaned['Start_Time'].dt.day_name()
city_day = df_cleaned.groupby(['City', 'Day_of_Week']).size().unstack(fill_value=0)

# normalize row-wise to get proportions per city
city_day_prop = city_day.div(city_day.sum(axis=1), axis=0)

# reorder columns to match actual day order
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
city_day_prop = city_day_prop[ordered_days]

# take top 20 cities by total accidents
top20_cities = df_cleaned['City'].value_counts().head(20).index
city_day_top20 = city_day_prop.loc[top20_cities]

# plot heatmap
plt.figure(figsize=(10, 10))
sb.heatmap(city_day_top20, cmap='YlGnBu', cbar_kws={'label': 'Proportion of Accidents'})
plt.title("Proportion of Accidents by Day in Top 20 Cities")
plt.xlabel("Day of Week")
plt.ylabel("City")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

The day trends are relevant to the cities with largest number of accidents, validating our focus on it. 

In [ ]:
# sort by total weekend proportion 
city_day_top20['Weekend'] = city_day_top20['Saturday'] + city_day_top20['Sunday']
city_day_top20_sorted = city_day_top20.sort_values(by='Weekend', ascending=False).drop(columns='Weekend')

# create annotation DataFrame with formatted strings
annot = (city_day_top20_sorted * 100).round(1).astype(str) + '%'
annot = annot.values  # Ensure it's a NumPy array

plt.figure(figsize=(10, 10))
sb.heatmap(city_day_top20_sorted, cmap='YlGnBu', cbar_kws={'label': 'Proportion of Accidents'}, 
            annot=annot, fmt='', annot_kws={"size": 9})
plt.title("Proportion of Accidents by Day in Top 20 Cities (Sorted by Weekend Share)")
plt.xlabel("Day of Week")
plt.ylabel("City")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

The above also validates that the trend is seen across all cities. Bakersfield has the most even distribution, but even it shows some differentiation. We will continue to look at this trend in our analysis.

The below shows us the weather analysis. We wish to see if all the accidents in specific streets occur in adverse conditions. This will assist in classifying if the streets are dangerous only in adverse weather, or if they are dangerous just in general. 

In [ ]:
# create Weather_Category based on Is_Clear flag
df['Weather_Category'] = df['Is_Clear'].apply(lambda x: 'Clear' if x == 1 else 'Not Clear')

# get top 20 streets by accident count
top_streets = df['Street'].value_counts().head(20).index

# filter to those streets
df_top_streets = df[df['Street'].isin(top_streets)].copy()

# createountplot
plt.figure(figsize=(12, 8))
ax = sb.countplot(
    data=df_top_streets,
    y='Street',
    hue='Weather_Category',
    order=top_streets,
    palette=['#6baed6', '#ff6f61']  # Clear = blue, Not Clear = red
)

# add percentages to the bars
# get counts grouped by street and weather
counts = (
    df_top_streets.groupby(['Street', 'Weather_Category'])
    .size()
    .unstack(fill_value=0)
)

# add annotation on bars
for i, street in enumerate(top_streets):
    total = counts.loc[street].sum()
    clear_count = counts.loc[street].get('Clear', 0)
    not_clear_count = counts.loc[street].get('Not Clear', 0)

    # get bar coordinates and add text
    bar_clear = ax.patches[i * 2]  # Each street has two bars (hue)
    bar_not_clear = ax.patches[i * 2 + 1]

plt.title("Accidents on Top 20 Streets by Weather Condition (Full Dataset)")
plt.xlabel("Number of Accidents")
plt.ylabel("Street")
plt.legend(title="Weather")
plt.tight_layout()
plt.show()

We see that unclear weather contributes heavily to the occurrence of accidents, making it a key factor that we wish to consider. The different factors of weather can be analysed to make predictions. For streets that show more evenness, we can exclude them.

To conclude, our three dimensions of region, time+day and weather are all validated. We narrow down specifically to investigate cities and streets for region, peak hour trends and weekday-weekend splits for time+day, and indicators of adverse weather. 

<h3>Machine Learning</h3>

Now that we have performed our EDA on the dataset, we understand a few key trends better. It is evident that there are a few key cities and regions that experience significantly higher volumes of accidents than others. We've also identified a trend throughout the day where accidents spike during peak hours and seem to be higher during the peak hours in the evening as opposed to the morning. 

Understanding these trends, we now want to apply our dataset into some Machine Learning applications. There were a few ideas that we considered as outputs from our algorithms.

1. Binary classification on whether a condition would result in accidents
2. Time series forecast to predict the rate of accidents in the next hour at a given hour, region and weather
3. Severity prediction for an accident based on time, region and weather

Unfortunately, as we are working with a positives-only dataset, it would be impossible to create an algorithm that could perform the binary classification we would want in 1 unless we merged our dataset with another. Furthermore, as seen above, only one unique value of severity of accident exists making it impossible to conduct classification on the severity. Thus, we will be going ahead to create algorithms for objectives 2 only.

Thus, let us tackle algorithm 2, predicting the rates of accidents in the upcoming hour based on the location, weather, time and day of the week.

To make the dataset easier to work with for our models, let us first remove all unnecessary columns. We will also work with cities first as aggregating by city would be easier to work with for the localised prediction.

In [ ]:
columns_to_keep = [
    'Start_Time',  # Time information
    'City', #City
    'Temperature(F)',  # Weather data
    'Humidity(%)',  # Weather data
    'Pressure(in)', #Pressure data
    'Wind_Speed(mph)',  # Weather data
    'Start_Hour',  # Extracted from Start_Time
    'Day_of_Week',  # Extracted from Start_Time
    'Is_Windy',
    'Is_Stormy',
    'Is_Rainy',
    'Is_Foggy',
    'Is_Snowy',
    'Is_Clear'
]

# Drop all other columns
df_algo1 = df_cleaned[columns_to_keep]

In [ ]:
df_algo1.head()

After reducing the dataset to focus on our key predictors, we now need to create the target variable for predicting the rate of accidents in the next hour.

In [ ]:
# Step 1: Floor to hour
df_algo1['Start_Hourly'] = df_algo1['Start_Time'].dt.floor('H')

# Step 2: Create "Next Hour"
df_algo1['Next_Hourly'] = df_algo1['Start_Hourly'] + pd.Timedelta(hours=1)

# Step 3: Count future accidents per city per hour
city_hourly_counts = df_algo1.groupby(['City', 'Start_Hourly']).size().rename('Accident_Count')

# Step 4: Shift within each city to get *next hour's* count
city_hourly_next = city_hourly_counts.groupby(level=0).shift(-1).rename('Accident_Next_Hour')

# Step 5: Combine the City/Hour index with the shifted counts
target_df = pd.concat([city_hourly_counts, city_hourly_next], axis=1).reset_index()

# Step 6: Merge back into main df
df_algo1 = df_algo1.merge(target_df[['City', 'Start_Hourly', 'Accident_Next_Hour']],
              on=['City', 'Start_Hourly'], how='left')


print(df_algo1.head(3))

In [ ]:
df_algo1.head()

In [ ]:
# Step 1: Make sure your time is rounded to the hour
df_algo1['Start_Hourly'] = df_algo1['Start_Time'].dt.floor('H')

# Step 2: Count number of accidents per city per hour
city_hourly_counts = df_algo1.groupby(['City', 'Start_Hourly']).size().rename('Accident_Current_Hour')

# Step 3: Create the lag (previous hour's accident count)
city_hourly_counts_lag = city_hourly_counts.groupby(level=0).shift(1).rename('Accident_Prev_Hour')

# Step 4: Combine the counts into a DataFrame for merging
lag_features = pd.concat([city_hourly_counts, city_hourly_counts_lag], axis=1).reset_index()

# Step 5: Merge lag features into main DataFrame
df_algo1 = df_algo1.merge(lag_features, how='left', on=['City', 'Start_Hourly'])

Since there is the cardinality of the City variable is very high, it would not be possible to convert it with One-Hot Encoding. Additionally, if label encoding is used, patterns about the distance between the numbers could be identified when in reality there is no pattern between the integers given to each city. Thus, we will use a different method called Target Encoding instead.

In [ ]:
# Calculate the mean of the target variable for each city
city_target_mean = df_algo1.groupby('City')['Accident_Next_Hour'].mean()

# Map the mean target value to the 'City' column
df_algo1['City_Encoded'] = df_algo1['City'].map(city_target_mean)

# Check the result
print(df_algo1.head(20))

Since the Day_of_Week currently exists as strings, we shall apply a simple ordinal encoding to allow it to be processed by the algorithms.

In [ ]:
day_of_week_map = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
}

# Apply the mapping to the 'Day_of_Week' column
df_algo1['Day_of_Week'] = df_algo1['Day_of_Week'].map(day_of_week_map)

# Check the result
print(df_algo1[['Day_of_Week']].head())

In [ ]:
# Drop rows where target is NaN
df_algo1 = df_algo1.dropna(subset=['Accident_Prev_Hour'])
df_algo1.info()

In [ ]:
df_algo1.head()

Finally, since the hours of the day are cyclical, let's convert our start_hour to represent the cyclical nature instead.

In [ ]:
# Hour of the day (0 to 23)
df_algo1['Start_Hour'] = df_algo1['Start_Time'].dt.hour

# Convert hour into sine and cosine to capture cyclic nature
df_algo1['Hour_Sin'] = np.sin(2 * np.pi * df_algo1['Start_Hour'] / 24)
df_algo1['Hour_Cos'] = np.cos(2 * np.pi * df_algo1['Start_Hour'] / 24)

df_algo1.head()

In [ ]:
df_algo1.describe()

In [ ]:
X = df_algo1[['Temperature(F)', 'Humidity(%)', 'Wind_Speed(mph)', 'Pressure(in)', 'Day_of_Week', 'City_Encoded', 'Is_Windy',
    'Is_Stormy',
    'Is_Rainy',
    'Is_Foggy',
    'Is_Snowy',
    'Is_Clear',
    'Hour_Sin', 'Hour_Cos',
    'Accident_Prev_Hour'] ]
y = df_algo1['Accident_Next_Hour']

from sklearn.model_selection import train_test_split
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

We will be testing 3 different algorithms to determine which will perform the best.

1. Random Forest Regressor
2. Gradient Boosting Regressor
3. Linear Regression
   
We will define hyperparameter grids to tune the hyperparameters for each algorithm.

In [ ]:
# Define the hyperparameters for Random Forest
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [ ]:
# Define the hyperparameters for Gradient Boosting
gb_param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [ ]:
# Define the hyperparameters for Ridge Regression (regularized Linear Regression)
lr_param_grid = {
    'alpha': [0.1, 1, 10, 100]
}

In [ ]:
def tune_model(model, param_grid, X_train, y_train, X_test, y_test, n_iter=1):
    search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=n_iter, cv=2,
                                random_state=42, n_jobs=-1)
    
    with tqdm(total=n_iter, desc=f"Training {model.__class__.__name__}") as pbar:
        search.fit(X_train, y_train)
        pbar.update(n_iter)

    best_model = search.best_estimator_
    y_pred = best_model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return best_model, mse, mae, r2, y_pred


In [ ]:

# --- Check and Handle NaN values in y_train ---
# This section is added to fix the "Input y contains NaN" error
print("--- NaN Handling ---")
initial_y_nans = y_train.isna().sum()
# Handle case where y_train might be a DataFrame (multi-output)
if isinstance(y_train, pd.DataFrame):
    initial_y_nans = initial_y_nans.sum()
print(f"Number of NaNs initially in y_train: {initial_y_nans}")

if initial_y_nans > 0:
    print(f"Original shape X_train: {X_train.shape}, y_train: {y_train.shape}")
    # Create a boolean mask for rows where y_train IS NOT NaN
    # If y_train is a DataFrame, .all(axis=1) keeps rows where *all* outputs are non-NaN.
    # Adjust to .any(axis=1) if needed, depending on your multi-output strategy.
    if isinstance(y_train, pd.DataFrame):
         mask = y_train.notna().all(axis=1)
    else: # Assumes y_train is a Pandas Series
         mask = y_train.notna()

    # Apply mask to remove rows with NaN in y_train from BOTH X_train and y_train
    X_train = X_train[mask]
    y_train = y_train[mask]

    # Confirm removal
    final_y_nans = y_train.isna().sum()
    if isinstance(y_train, pd.DataFrame): final_y_nans = final_y_nans.sum()
    print(f"New shape after removing NaN y_train rows: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Number of NaNs remaining in y_train: {final_y_nans}")
else:
    print("No NaNs found in y_train.")
print("--- End NaN Handling ---")

# --- Check for NaNs in X_train (Good Practice) ---
# Note: Handling X_train NaNs (e.g., imputation) should ideally happen earlier.
initial_x_nans = X_train.isna().sum().sum()
print(f"Total NaNs found in X_train: {initial_x_nans}")
if initial_x_nans > 0:
    print("Warning: NaNs detected in X_train. Ensure they are handled appropriately (e.g., imputation).")
print("-" * 20)

# --- Initialise models ---
# (Your original initialization)
rf_model = RandomForestRegressor(random_state=42)
gb_model = GradientBoostingRegressor(random_state=42)
lr_model = Ridge()

# --- Train and collect metrics + predictions ---
# Added n_iter parameter (adjust value as needed for RandomizedSearchCV)
tuning_iterations = 10 # Example: test 10 random parameter combinations per model

# Ensure your tune_model function accepts and uses the n_iter argument
rf_best_model, rf_mse, rf_mae, rf_r2, rf_pred = tune_model(rf_model, rf_param_grid, X_train, y_train, X_test, y_test, n_iter=tuning_iterations)
gb_best_model, gb_mse, gb_mae, gb_r2, gb_pred = tune_model(gb_model, gb_param_grid, X_train, y_train, X_test, y_test, n_iter=tuning_iterations)
# Note: RandomizedSearch might be less common for Ridge, but using for consistency.
# Adjust n_iter or use GridSearchCV via tune_model if lr_param_grid is small.
lr_best_model, lr_mse, lr_mae, lr_r2, lr_pred = tune_model(lr_model, lr_param_grid, X_train, y_train, X_test, y_test, n_iter=tuning_iterations)

# --- Create a results table ---
# Added checks to ensure models trained successfully before adding results
# (Assumes tune_model returns None or np.inf/nan on failure - adjust if needed)
results = {}
if rf_best_model is not None and np.isfinite(rf_mse):
     results['Random Forest'] = {'MSE': rf_mse, 'MAE': rf_mae, 'R²': rf_r2}
if gb_best_model is not None and np.isfinite(gb_mse):
     results['Gradient Boosting'] = {'MSE': gb_mse, 'MAE': gb_mae, 'R²': gb_r2}
if lr_best_model is not None and np.isfinite(lr_mse):
    # Changed key slightly for clarity
    results['Linear Regression (Ridge)'] = {'MSE': lr_mse, 'MAE': lr_mae, 'R²': lr_r2}

# Only create and print DataFrame if results dictionary is not empty
if results:
    results_df = pd.DataFrame(results).T
    print("\n--- Model Performance ---")
    print(results_df.round(4))
else:
    print("\nNo models trained successfully or yielded valid metrics.")

In [ ]:
# Plot Actual vs Predicted
def plot_actual_vs_pred(y_true, y_pred, title):
    plt.figure(figsize=(8, 5))
    plt.scatter(y_true, y_pred, alpha=0.3, edgecolor='k')
    plt.xlabel("Actual Accident Count")
    plt.ylabel("Predicted Accident Count")
    plt.title(f"Actual vs Predicted: {title}")
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
    plt.grid(True)
    plt.show()

# Residuals
def plot_residuals(y_true, y_pred, title):
    residuals = y_true - y_pred
    plt.figure(figsize=(8, 4))
    plt.hist(residuals, bins=50, alpha=0.7, color='purple')
    plt.title(f"Residuals: {title}")
    plt.xlabel("Prediction Error")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

# Run for each model
plot_actual_vs_pred(y_test, rf_pred, "Random Forest")
plot_residuals(y_test, rf_pred, "Random Forest")

plot_actual_vs_pred(y_test, gb_pred, "Gradient Boosting")
plot_residuals(y_test, gb_pred, "Gradient Boosting")

plot_actual_vs_pred(y_test, lr_pred, "Linear Regression")
plot_residuals(y_test, lr_pred, "Linear Regression")

In [ ]:
df_algo1['Accident_Next_Hour'].hist(bins=30)

In [ ]:
def categorize_accidents(x):
    if x == 0:
        return 'none'
    elif x <= 2:
        return 'low'
    elif x <= 5:
        return 'medium'
    else:
        return 'high'

df_algo1['Accident_Class'] = df_algo1['Accident_Next_Hour'].apply(categorize_accidents)
df_algo1.head()

In [ ]:
# Define feature columns
feature_cols = [
    'Temperature(F)', 'Humidity(%)', 'Wind_Speed(mph)', 'Pressure(in)',
    'City_Encoded', 'Is_Windy', 'Is_Stormy', 'Is_Rainy',
    'Is_Foggy', 'Is_Snowy', 'Is_Clear',
    'Hour_Sin', 'Hour_Cos', 'Accident_Prev_Hour'
]

# Encode target classes
le = LabelEncoder()
y_encoded = le.fit_transform(df_algo1['Accident_Class'])
# Define features and target
X = df_algo1[feature_cols]
y = y_encoded

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Apply SMOTE to the training data
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train, y_train)

# print("Before SMOTE:", y_train.value_counts())
# print("After SMOTE:", pd.Series(y_train_resampled).value_counts())

In [ ]:
# Define classifiers
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced'),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    'HistGradientBoosting': HistGradientBoostingClassifier(random_state=42)
}

results = {}

for name, model in models.items():
    print(f"\n🧪 Training {name}...")
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test)

    print(classification_report(y_test, y_pred))

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
    disp.plot(cmap='Blues')
    plt.title(f"Confusion Matrix: {name}")
    plt.grid(False)
    plt.show()

    # Save overall accuracy and macro F1
    from sklearn.metrics import accuracy_score, f1_score
    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "F1 (macro)": f1_score(y_test, y_pred, average='macro')
    }

In [ ]:
feature_names = X.columns  # assuming X was the feature DataFrame used

# Access the already-trained Random Forest model
rf_model = models['Random Forest']

# Check if the model supports feature importances
if hasattr(rf_model, 'feature_importances_'):
    importances = rf_model.feature_importances_

    # Create a DataFrame of features and their importance
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)

    # Display top 10
    print("🔍 Top 10 Most Important Features:\n")
    print(importance_df.head(10))

    # Plot
    plt.figure(figsize=(10, 6))
    plt.barh(importance_df['Feature'][:10][::-1], importance_df['Importance'][:10][::-1])
    plt.xlabel("Importance")
    plt.title("Top 10 Feature Importances - Random Forest")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("❌ This model does not support feature importances.")

<h3> Data-Driven Insights </h3>

With the classification model, we understand that ‘City’ has the highest prediction importance at 0.211, followed by ‘Humidity’ (0.151) , ‘Pressure’ (0.149), ‘Temperature’ (0.148), ‘Windspeed’ (0.112), ‘Accident In Previous Hour’ (0.099) and ‘Hour’ (0.058). 

This supports that Los Angeles, Sacramento, San Diego and San Jose would have a significantly higher possibility of accidents relative to other cities. The number of accidents drops sharply after these four cities, making it harder to make definitive predictions for the other cities. 

Next, the series of climate-related metrics that are all interrelated, allude to adverse weather. High humidity is possibly an indicator of precipitation, but the correlation to temperature possibly links to driver comfort as well. 

‘Accidents in Previous Hour’ shows high clustering of accidents, and ‘Hour’ supports our peak hour hypothesis. 

Therefore, to interpret the outputs exactly as an example, we would be at our highest caution when driving in Los Angeles at 5PM on a Friday, especially when it is raining in the summer and there have been other accidents in the vicinity.

The model also warns that it shows strong predictive accuracy for low and moderate severity accidents, especially severity 1 and 2 — with recall scores of 0.83 and 0.74 respectively. However, it struggles to distinguish between severity 3 and 4 accidents due to overlapping environmental features. This is evident in the confusion matrix where 47% of severity 4 accidents are misclassified. 

Therefore, we have to remember the caveat that while these predictions will help us with general accidents, freak catastrophic accidents can still very much occur in any situation, without following these trends. This keeps us alert and prevents any complacency in driving. 
